<h1 align="center">Projet IA HumanForYou</h1>

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import warnings
import tarfile
import os

# Configuring display settings
plt.rcParams['figure.figsize'] = (12, 5)
sns.set()
sns.set_context('talk')
np.set_printoptions(threshold=20, precision=2, suppress=True)
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)
pd.set_option('display.float_format', '{:.2f}'.format) 
warnings.filterwarnings("ignore", category=FutureWarning)

In [5]:
general_df = pd.read_csv("Data_cleaned/general_data.csv")
manager_srv_df = pd.read_csv("Data_cleaned/manager_survey_data.csv")
employee_srv_df = pd.read_csv("Data_cleaned/employee_survey_data.csv")
horaires_df = pd.read_csv("Data_cleaned/features_horaires_travail.csv")

In [6]:
# Fusion general + manager survey
merged_df = pd.merge(general_df, manager_srv_df, on='EmployeeID', how='left')

# Fusion avec employee survey
merged_df = pd.merge(merged_df, employee_srv_df, on='EmployeeID', how='left')

# Fusion avec les horaires agrégés
merged_df = pd.merge(merged_df, horaires_df, on='EmployeeID', how='left')


In [11]:
print(merged_df.shape)         # Vérifier la taille du dataset fusionné
print(merged_df.info())        # Vérifier les colonnes et types
print(merged_df.isnull().sum())# Vérifier s’il reste des valeurs manquantes


(4410, 32)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 32 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       4410 non-null   int64  
 1   Attrition                 4410 non-null   object 
 2   BusinessTravel            4410 non-null   object 
 3   Department                4410 non-null   object 
 4   DistanceFromHome          4410 non-null   int64  
 5   Education                 4410 non-null   int64  
 6   EducationField            4410 non-null   object 
 7   EmployeeCount             4410 non-null   int64  
 8   EmployeeID                4410 non-null   int64  
 9   Gender                    4410 non-null   object 
 10  JobLevel                  4410 non-null   int64  
 11  JobRole                   4410 non-null   object 
 12  MaritalStatus             4410 non-null   object 
 13  MonthlyIncome             4410 non-null   int64  
 1

In [12]:
merged_df.to_csv("Data_cleaned/all_data.csv", index=False)
